# 6. 19대 대선 결과 분석

## 1. Selenium과 Beautiful Soup을 이용한 데이터 획득 준비 작업

In [17]:
import pandas as pd
import numpy as np
import platform
import matplotlib.pyplot as plt 
%matplotlib inline

In [18]:
import platform
from matplotlib import font_manager, rc

plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
else:
    print('Unknown System..')

In [19]:
from selenium import webdriver
import time

In [20]:
driver = webdriver.Chrome('../driver/chromedriver')
driver.get("http://info.nec.go.kr/main/showDocument.xhtml?electionId=0000000000&topMenuId=VC&secondMenuId=VCCP09")

In [21]:
driver.find_element_by_id('electionType1').click()

In [22]:
driver.find_element_by_id('electionName').send_keys('제19대')

In [23]:
driver.find_element_by_id('electionCode').send_keys('대통령선거')

In [31]:
sido_list_raw = driver.find_element_by_xpath("""//*[@id="cityCode"]""")
sido_list = sido_list_raw.find_elements_by_tag_name('option')
sido_names_values = [option.text for option in sido_list]
sido_names_values = sido_names_values[2:]
sido_names_values

['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원도',
 '충청북도',
 '충청남도',
 '전라북도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

## 2. 19대 대선 개표 결과 데이터 획득하기

In [32]:
# '득표수(득표율)'에서 '(' 기준으로 왼쪽 숫자만 얻어서 콤마 제거하고 float형으로 변경하는 함수
import re
def get_num(tmp):
    return float(re.split('\(', tmp)[0].replace(',',''))

In [33]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 

wait = WebDriverWait(driver, 10)

# move_sido 함수는 광역시도 이름을 리스트에 전송하고 검색 버튼을 누르는 역할
def move_sido(name):
    element = driver.find_element_by_id('cityCode')
    element.send_keys(name)
    make_xpath = """//*[@id="searchBtn"]"""
    wait.until(EC.element_to_be_clickable((By.XPATH, make_xpath)))
    driver.find_element_by_xpath(make_xpath).click()

In [41]:
# append_data 함수는 빈 내용을 미리 준비한 DataFrame에 append 명령으로 읽은 데이터를 하나씩 추가하는 기능
def append_data(df, sido_name, data):
    for each in df[0].values[1:]:
        data['광역시도'].append(sido_name)
        data['시군'].append(each[0])
        data['pop'].append(each[2])
        data['moon'].append(get_num(each[3]))
        data['hong'].append(get_num(each[4]))
        data['ahn'].append(get_num(each[5]))

In [44]:
# 미리 변수를 하나 만든다
election_result_raw = {
    '광역시도' : [],
    '시군' : [],
    'pop' : [],
    'moon' : [],
    'hong' : [],
    'ahn' : []
}

In [45]:
# html 태그 중 table로 되어 있는 부분을 Beautiful Soup으로 읽어서 pandas의 read_html로 읽는다.
from bs4 import BeautifulSoup

for each_sido in sido_names_values:
    move_sido(each_sido)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table')
    df = pd.read_html(str(table))
    append_data(df, each_sido, election_result_raw)

In [46]:
election_result = pd.DataFrame(election_result_raw, columns=['광역시도', '시군', 'pop', 'moon', 'hong', 'ahn'])
election_result

,광역시도,시군,pop,moon,hong,ahn
0,서울특별시,종로구,102566,42512.0,22325.0,22313.0
1,서울특별시,중구,82852,34062.0,17901.0,19372.0
2,서울특별시,용산구,148157,58081.0,35230.0,32109.0
3,서울특별시,성동구,203175,86686.0,40566.0,45674.0
4,서울특별시,광진구,240030,105512.0,46368.0,52824.0
5,서울특별시,동대문구,236092,98958.0,51631.0,53359.0
6,서울특별시,중랑구,265706,111450.0,56545.0,62778.0
7,서울특별시,성북구,295866,129263.0,57584.0,66518.0
8,서울특별시,강북구,210614,89645.0,42268.0,51669.0
9,서울특별시,도봉구,229233,94898.0,47461.0,55600.0


In [47]:
# 저장하기
election_result.to_csv('../data/05. election_result.csv', sep=',')

In [48]:
driver.close()

## 3. 각 후보의 득표율과 지역 ID 정리하기